In [127]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, Dropout, Dense
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K
from tensorflow.keras.applications import VGG16, MobileNetV3Small, MobileNetV2
from tensorflow.keras.losses import BinaryFocalCrossentropy, BinaryCrossentropy



print("TensorFlow Version: ", tf.__version__)


TensorFlow Version:  2.8.0


In [5]:
#MobileNetV3Small().save('mobilenetv3.h5')
# VGG16().save('vgg16.h5')
MobileNetV2().save('mobilenetv2.h5')


# Data loading


In [52]:
np.random.seed(42)
tf.random.set_seed(42)

IMAGE_SIZE = 256
EPOCHS = 30
BATCH = 8
LR = 1e-4

PATH = os.path.join("dataset", "CVC-ClinicDB")


In [50]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))
    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [43]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

In [45]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [53]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(PATH)

print("Training data: ", len(train_x))
print("Validation data: ", len(valid_x))
print("Testing data: ", len(test_x))

lOADED
Total size : 612
Training data:  490
Validation data:  61
Testing data:  61


In [54]:
def read_and_rgb(x):
    x = cv2.imread(x)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    return x

# Mobilenetv2 Unet - Mobile semantic segmentation neural network

In [15]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [21]:
def decoder_block(input, skip_features, num_filters, strides):
    x = Conv2DTranspose(num_filters, (2, 2), strides=strides, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [89]:
def build_mobilenetv3_unet(input_shape):
    inputs = Input(input_shape, name="input_image")
    mobilenetv3 = MobileNetV3Small(include_top=False, weights='imagenet', input_tensor=inputs)
    """ Pre-trained MobileNetv2"""
    skip_connection_names = ["input_image", "re_lu", "re_lu_1", "re_lu_6", "re_lu_13"]

    encoder_output = mobilenetv3.get_layer('re_lu_25').output
    feature_map = [16, 32, 48, 64, 128]
    x = encoder_output
    for i in range(1, len(skip_connection_names)+1, 1):
        print("[*] {}".format(skip_connection_names[-i]))
        x_skip = mobilenetv3.get_layer(skip_connection_names[-i]).output     

        x = UpSampling2D((2, 2))(x)
        print("X {} Xskip {} ".format(x, x_skip))

        x = Concatenate()([x, x_skip])
        
        x = Conv2D(feature_map[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
        x = Conv2D(feature_map[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

    x = Dropout(0.1)(x)
    x = Dense(512, kernel_regularizer='l1')(x)
    # last layer for segmentation 
    x = Conv2D(1, (1,1), padding="same")(x)
    x = Activation("sigmoid")(x)
    model = Model(inputs, x)
    print("[*] Done")
    return model


In [91]:
model = MobileNetV2(include_top=False, input_tensor=Input((256, 256, 3)))
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [123]:
input_shape=(256, 256, 3)
model = build_mobilenetv3_unet(input_shape)
model.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 256, 256, 3)  0           ['input_image[0][0]']            
                                                                                                  
 Conv (Conv2D)                  (None, 128, 128, 16  432         ['rescaling_1[0][0]']            
                                )                                                                 
                                                                                   

In [119]:
def build_mobilenetv2_unet(input_shape, alpha=0.35):
    """ input """
    print("[*] Building model ..")
    inputs = Input(input_shape, name="input_image")

    """ Pre-trained MobileNetv2"""
    mobilenetv2 = MobileNetV2(include_top=False, weights='imagenet', input_tensor = inputs, alpha=alpha)
    skip_connection_names = ["input_image", "block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu", "block_10_expand_relu"]

    encoder_output = mobilenetv2.get_layer('block_16_expand_relu').output
    feature_map = [16, 32, 48, 64, 80]
    x = encoder_output
    for i in range(1, len(skip_connection_names)+1, 1):
        print("[*] {}".format(skip_connection_names[-i]))
        x_skip = mobilenetv2.get_layer(skip_connection_names[-i]).output     

        x = UpSampling2D((2, 2))(x)
        print("X {} Xskip {} ".format(x, x_skip))

        x = Concatenate()([x, x_skip])
        
        x = Conv2D(feature_map[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
        x = Conv2D(feature_map[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

    x = Dropout(0.1)(x)
    x = Dense(512, kernel_regularizer='l1')(x)
    # last layer for segmentation 
    x = Conv2D(1, (1,1), padding="same")(x)
    x = Activation("sigmoid")(x)
    model = Model(inputs, x)
    print("[*] Done")
    return model
    

In [117]:
input_shape=(256, 256, 3)
model = build_mobilenetv2_unet(input_shape)
model.summary()

[*] Building model ..


ValueError: If imagenet weights are being loaded, alpha must be one of `0.35`, `0.50`, `0.75`, `1.0`, `1.3` or `1.4` only; Received `alpha=0.3`

# Metrics & Training

In [64]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def iou(y_true,y_pred):
  def f(y_true,y_pred):
    intersection = (y_true*y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    x = (intersection + 1e-15) / (union + 1e-15)
    x = x.astype(np.float32)
    return x
  return tf.numpy_function(f,[y_true,y_pred],tf.float32)

In [132]:
train_dataset = tf_dataset(train_x, train_y, batch=BATCH)
valid_dataset = tf_dataset(valid_x, valid_y, batch=BATCH)
opt = tf.keras.optimizers.Nadam(LR)
metrics = [ Recall(), Precision(), iou]

input_shape=(256, 256, 3)
model = build_mobilenetv2_unet(input_shape, alpha=0.5)
model.summary()

model.compile(loss=BinaryFocalCrossentropy(), optimizer=opt, metrics=metrics)

[*] Building model ..
[*] block_10_expand_relu
X KerasTensor(type_spec=TensorSpec(shape=(None, 16, 16, 336), dtype=tf.float32, name=None), name='up_sampling2d_116/resize/ResizeNearestNeighbor:0', description="created by layer 'up_sampling2d_116'") Xskip KerasTensor(type_spec=TensorSpec(shape=(None, 16, 16, 144), dtype=tf.float32, name=None), name='block_10_expand_relu/Relu6:0', description="created by layer 'block_10_expand_relu'") 
[*] block_6_expand_relu
X KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 80), dtype=tf.float32, name=None), name='up_sampling2d_117/resize/ResizeNearestNeighbor:0', description="created by layer 'up_sampling2d_117'") Xskip KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 96), dtype=tf.float32, name=None), name='block_6_expand_relu/Relu6:0', description="created by layer 'block_6_expand_relu'") 
[*] block_3_expand_relu
X KerasTensor(type_spec=TensorSpec(shape=(None, 64, 64, 64), dtype=tf.float32, name=None), name='up_sampling2d_118/resize/ResizeN

In [125]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

In [133]:
train_steps = len(train_x)//BATCH
valid_steps = len(valid_x)//BATCH

if len(train_x) % BATCH != 0:
    train_steps += 1
if len(valid_x) % BATCH != 0:
    valid_steps += 1

model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)

Epoch 1/30


2022-05-06 11:58:12.758546: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


62/62 [==============================] - ETA: 0s - loss: 4.2615 - recall_9: 0.6030 - precision_9: 0.2644 - iou: 0.0996

2022-05-06 12:01:13.212940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


62/62 [==============================] - 193s 1s/step - loss: 4.2615 - recall_9: 0.6030 - precision_9: 0.2644 - iou: 0.0996 - val_loss: 4.0351 - val_recall_9: 0.8509 - val_precision_9: 0.1925 - val_iou: 0.0961 - lr: 1.0000e-04
Epoch 2/30
62/62 [==============================] - 42s 639ms/step - loss: 3.7223 - recall_9: 0.6069 - precision_9: 0.8558 - iou: 0.1304 - val_loss: 3.5008 - val_recall_9: 0.3917 - val_precision_9: 0.7657 - val_iou: 0.0978 - lr: 1.0000e-04
Epoch 3/30
62/62 [==============================] - 38s 614ms/step - loss: 3.2439 - recall_9: 0.7010 - precision_9: 0.9170 - iou: 0.1534 - val_loss: 3.0363 - val_recall_9: 0.2609 - val_precision_9: 0.9781 - val_iou: 0.0965 - lr: 1.0000e-04
Epoch 4/30
62/62 [==============================] - 38s 608ms/step - loss: 2.8086 - recall_9: 0.7752 - precision_9: 0.9422 - iou: 0.1728 - val_loss: 2.6180 - val_recall_9: 0.2820 - val_precision_9: 0.9785 - val_iou: 0.1031 - lr: 1.0000e-04
Epoch 5/30
62/62 [==============================] - 3

In [84]:
model.save("unet-vanilla-0_17-0_94-0_96-0_98-0-98-val-iou-0_69")

INFO:tensorflow:Assets written to: unet-vanilla-0_17-0_94-0_96-0_98-0-98-val-iou-0_69/assets


# Evaluation

In [ ]:
test_dataset = tf_dataset(test_x, test_y, batch=BATCH)

test_steps = (len(test_x)//BATCH)
if len(test_x) % BATCH != 0:
    test_steps += 1

model.evaluate(test_dataset, steps=test_steps)